# Quiz 소프트웨어활용및코딩Q7-데이터야 놀자

In [1]:
import pandas as pd
url = '../exampleCode/dataset/kopo_channel_seasonality_new.csv'
selloutData = pd.read_csv(url)
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


### 1. A01지역의 2015년도 제품별 평균판매량이 가장 높은 상품 3개는? (콤마로 구분하여 작성, 예: PRODUCT12,PRODUCT2,PRODUCT3)

In [2]:
# 연주차 분리
def yearweekSeparation(inValue, option):
    # inValue = 201402
    inValue = str(inValue)
    
    # 1년, 999년, 10000년에 대응하기 위해서 마이너스 인덱스 사용.
    year = inValue[:-2]
    week = inValue[-2:]
    if option =="year":
        value = year
    else:
        value = week
    return value

In [3]:
selloutData["YEAR"] = selloutData.YEARWEEK.apply(yearweekSeparation, option = "year")
selloutData["WEEK"] = selloutData.YEARWEEK.apply(yearweekSeparation, option = "")
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,2014,02
...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,2016,30


In [4]:
# 2015년도 A01 지역으로 한정하기
selloutData2015_A01 = selloutData[(selloutData.REGIONID == 'A01') & (selloutData.YEAR == '2015')]
selloutData2015_A01

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
22730,A01,PRODUCT47,201512,5333.0,2015,12
22731,A01,PRODUCT60,201512,0.0,2015,12
22732,A01,PRODUCT56,201512,5151.0,2015,12
22733,A01,PRODUCT57,201512,7488.0,2015,12
22734,A01,PRODUCT12,201512,151750.0,2015,12
...,...,...,...,...,...,...
120742,A01,PRODUCT12,201547,378339.0,2015,47
120743,A01,PRODUCT1,201547,8214.0,2015,47
120744,A01,PRODUCT33,201547,17276.0,2015,47
120745,A01,PRODUCT14,201547,32572.0,2015,47


In [5]:
# 'PRODUCT' 컬럼으로 그룹화하여 그 기준으로 'QTY'를 sum 해서 새 컬럼에 담아라.
# groupKey = ['REGIONID','PRODUCT','YEARWEEK','YEAR','WEEK'] 이렇게 하면 PRODUCT만으로 그룹화가 안 된다. DB에서 PK라 보면 된다.
groupKey = ['PRODUCT']
groupData2015_A01 = selloutData2015_A01.groupby(by=groupKey)['QTY'].agg(['sum'])
groupData2015_A01.reset_index(inplace=True)
groupData2015_A01

,PRODUCT,sum
0,PRODUCT1,338342.0
1,PRODUCT12,10397477.0
2,PRODUCT14,992474.0
3,PRODUCT15,256580.0
4,PRODUCT16,76504.0
5,PRODUCT19,189018.0
6,PRODUCT2,1034242.0
7,PRODUCT20,3385.0
8,PRODUCT33,596078.0
9,PRODUCT34,5892319.0


In [7]:
# inplace를 했을 때 warning이 발생. 실제 에러는 아니고 경고이므로 보기 싫으면 이렇게 넣어주면 된다.
import warnings

warnings.filterwarnings(action="ignore")

In [7]:
# 'sum'을 기준으로 정렬하기
sortKey = ['sum']
groupData2015_A01.sort_values(by=sortKey, inplace=True, ascending=False)
groupData2015_A01.head(3)

,PRODUCT,sum
15,PRODUCT59,34620159.0
1,PRODUCT12,10397477.0
9,PRODUCT34,5892319.0


### 2. 2015년도 A01 지역에서 생산한 제품의 종류는 몇종인가요?

In [8]:
selloutData2015_A01 = selloutData2015_A01.loc[ : , ['PRODUCT']].drop_duplicates()
# selloutData2015A01

In [9]:
len(selloutData2015_A01)

23

In [10]:
len(groupData2015_A01)    # 1번에서 'PRODUCT'로 groupby를 했기 때문에 이렇게 구해도 된다.

23

### 3. 2015년도 1사분기 (1~13주) 평균 판매량이 가장 높은 지역 5개를 순서대로 적어주세요 (예: A01, A02,A03,A04,A05)

In [11]:
selloutData2015_1Q = selloutData[(selloutData.YEAR == '2015') & (selloutData.WEEK <= '13')]
selloutData2015_1Q

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
22005,A47,PRODUCT2,201512,2.0,2015,12
22006,A47,PRODUCT58,201512,3546.0,2015,12
22007,A47,PRODUCT59,201512,148781.0,2015,12
22008,A47,PRODUCT34,201512,29332.0,2015,12
22009,A48,PRODUCT35,201512,4611.0,2015,12
...,...,...,...,...,...,...
90063,A25,PRODUCT14,201510,0.0,2015,10
90064,A25,PRODUCT62,201510,564.0,2015,10
90065,A25,PRODUCT17,201510,115.0,2015,10
90066,A28,PRODUCT58,201510,139.0,2015,10


In [12]:
# 'REGIONID' 컬럼으로 그룹화하여 그 기준으로 'QTY'를 mean 해서 새 컬럼에 담아라.
groupKey = ['REGIONID']
groupData2015_1Q = selloutData2015_1Q.groupby(by=groupKey)['QTY'].agg(['mean'])
groupData2015_1Q.reset_index(inplace=True)

In [13]:
sortKey = ['mean']
groupData2015_1Q.sort_values(by=sortKey, inplace=True, ascending=False)
groupData2015_1Q.head()

,REGIONID,mean
40,A41,154823.096154
46,A47,52446.076923
1,A01,40121.414716
29,A29,29359.717949
27,A27,28716.201923


### 4. 생각해볼 문제 (지역, 상품, 분기별 평균 판매량을 산출하세요) 분기는 1-13, 14-26, 27-39, 40-52 로 구분 (53 제외)

In [14]:
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,2014,02
1,A60,PRODUCT59,201402,22275.0,2014,02
2,A60,PRODUCT34,201402,4463.0,2014,02
3,A60,PRODUCT47,201402,0.0,2014,02
4,A60,PRODUCT56,201402,23.0,2014,02
...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,2016,30
124654,A10,PRODUCT56,201630,275.0,2016,30
124655,A10,PRODUCT61,201630,0.0,2016,30
124656,A10,PRODUCT12,201630,15021.0,2016,30


In [15]:
selloutData = selloutData[(selloutData.WEEK <= '52')]

In [16]:
# 분기 분리
def defineQuater(inValue):
    # inValue = 14
    inValue = int(inValue)
    quater = ""
    
    # 1분기 1~13, 2분기 14~26, 3분기 27~39, 4분기 40~52
    if (inValue <= 13):
        quater = "1Q"
    elif (inValue <= 26):
        quater = "2Q"
    elif (inValue <= 39):
        quater = "3Q"
    elif (inValue <= 52):
        quater = "4Q"
    else:
        pass
    return quater

In [17]:
selloutData["QUATER"] = selloutData.WEEK.apply(defineQuater)
selloutDataQuater = selloutData

In [18]:
selloutDataQuater

,REGIONID,PRODUCT,YEARWEEK,QTY,YEAR,WEEK,QUATER
0,A60,PRODUCT4,201402,71.0,2014,02,1Q
1,A60,PRODUCT59,201402,22275.0,2014,02,1Q
2,A60,PRODUCT34,201402,4463.0,2014,02,1Q
3,A60,PRODUCT47,201402,0.0,2014,02,1Q
4,A60,PRODUCT56,201402,23.0,2014,02,1Q
...,...,...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,2016,30,3Q
124654,A10,PRODUCT56,201630,275.0,2016,30,3Q
124655,A10,PRODUCT61,201630,0.0,2016,30,3Q
124656,A10,PRODUCT12,201630,15021.0,2016,30,3Q


In [19]:
# 지역별 평균 판매량

In [20]:
# 'REGIONID' 컬럼으로 그룹화하여 그 기준으로 'QTY'를 mean 해서 새 컬럼에 담아라.
groupKey = ['REGIONID']
selloutDataRegionid = selloutDataQuater # selloutDataQuater가 변형되는 것을 방지하기 위해 새 변수에 담아준다. (selloutDataQuater 변수 재사용을 위해)
selloutDataRegionid = selloutDataRegionid.groupby(by=groupKey)['QTY'].agg(['mean'])
selloutDataRegionid.reset_index(inplace=True)
selloutDataRegionid.head()

,REGIONID,mean
0,A00,12065.292735
1,A01,51483.457079
2,A02,5548.169683
3,A03,8935.041478
4,A04,7174.930199


In [21]:
sortKey = ['mean']
selloutDataRegionid.sort_values(by=sortKey, inplace=True, ascending=False)
selloutDataRegionid.head()

,REGIONID,mean
40,A41,139001.642628
46,A47,55687.668269
1,A01,51483.457079
27,A27,29994.596955
20,A20,29179.055556


In [22]:
# 상품별 평균 판매량

In [23]:
# 'PRODUCT' 컬럼으로 그룹화하여 그 기준으로 'QTY'를 mean 해서 새 컬럼에 담아라.
groupKey = ['PRODUCT']
selloutDataProduct = selloutDataQuater
selloutDataProduct = selloutDataProduct.groupby(by=groupKey)['QTY'].agg(['mean'])
selloutDataProduct.reset_index(inplace=True)
selloutDataProduct.head()

,PRODUCT,mean
0,PRODUCT1,743.202051
1,PRODUCT10,2.288462
2,PRODUCT12,14624.447001
3,PRODUCT13,43.990385
4,PRODUCT14,721.751326


In [24]:
sortKey = ['mean']
selloutDataProduct.sort_values(by=sortKey, inplace=True, ascending=False)
selloutDataProduct.head()

,PRODUCT,mean
38,PRODUCT59,76782.239725
52,PRODUCT75,15552.506410
2,PRODUCT12,14624.447001
18,PRODUCT34,7887.551186
9,PRODUCT2,7586.240741


In [25]:
# 분기별 평균 판매량

In [26]:
# 'PRODUCT' 컬럼으로 그룹화하여 그 기준으로 'QTY'를 mean 해서 새 컬럼에 담아라.
groupKey = ['QUATER']
selloutDataQuaterNew = selloutDataQuater
selloutDataQuaterNew = selloutDataQuaterNew.groupby(by=groupKey)['QTY'].agg(['mean'])
selloutDataQuaterNew.reset_index(inplace=True)
selloutDataQuaterNew.head()

,QUATER,mean
0,1Q,7635.595621
1,2Q,8332.657191
2,3Q,9042.678357
3,4Q,10713.418640


# 수업

In [1]:
# 1. 데이터 불러오고 확인하기.
import pandas as pd

url = '../exampleCode/dataset/kopo_channel_seasonality_new.csv'
selloutData = pd.read_csv(url)

selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [2]:
selloutData.shape

(124658, 4)

In [3]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
dtype: object

In [4]:
preFix = '1207_'

In [5]:
def monthCvt(inColumn, appe):
    # 디버깅 코드
#     inColumn = "201402"

    inWeek = int(str(inColumn)[4:])
    outValue = "다른월"

    if inWeek < 5:
        outValue = "1월"
    elif inWeek < 10:
        outValue = "2월"
    else:
        outValue = "다른월"
        
    outValue = appe + outValue
    return outValue

In [6]:
selloutData['MONTH'] = selloutData.YEARWEEK.apply(monthCvt, appe = preFix)

In [7]:
selloutData

,REGIONID,PRODUCT,YEARWEEK,QTY,MONTH
0,A60,PRODUCT4,201402,71.0,1207_1월
1,A60,PRODUCT59,201402,22275.0,1207_1월
2,A60,PRODUCT34,201402,4463.0,1207_1월
3,A60,PRODUCT47,201402,0.0,1207_1월
4,A60,PRODUCT56,201402,23.0,1207_1월
...,...,...,...,...,...
124653,A10,PRODUCT60,201630,824.0,1207_다른월
124654,A10,PRODUCT56,201630,275.0,1207_다른월
124655,A10,PRODUCT61,201630,0.0,1207_다른월
124656,A10,PRODUCT12,201630,15021.0,1207_다른월


In [8]:
selloutData.loc[(selloutData.YEARWEEK.astype(str).str[4:].astype(int) < 5), "MONTH2"] = 1
selloutData.loc[(selloutData.YEARWEEK.astype(str).str[4:].astype(int) >= 5) &
                (selloutData.YEARWEEK.astype(str).str[4:].astype(int) < 10), "MONTH2"] = 2

In [9]:
selloutData[selloutData.MONTH2 == 2]

,REGIONID,PRODUCT,YEARWEEK,QTY,MONTH,MONTH2
230,A12,PRODUCT34,201405,1267.0,1207_2월,2.0
231,A12,PRODUCT40,201405,26.0,1207_2월,2.0
232,A12,PRODUCT47,201405,13.0,1207_2월,2.0
233,A12,PRODUCT60,201405,317.0,1207_2월,2.0
234,A12,PRODUCT56,201405,42.0,1207_2월,2.0
...,...,...,...,...,...,...
121658,A25,PRODUCT56,201608,147.0,1207_2월,2.0
121659,A25,PRODUCT57,201608,1473.0,1207_2월,2.0
121660,A25,PRODUCT12,201608,5846.0,1207_2월,2.0
121661,A25,PRODUCT1,201608,9.0,1207_2월,2.0


### 5-1 데이터 분석 및 전처리 P.77

In [1]:
import pandas as pd
url = '../exampleCode/dataset/kopo_product_volume.csv'
selloutData = pd.read_csv(url)
selloutData.head()

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME
0,A01,ST0001,201415,810144
1,A01,ST0002,201415,128999
2,A01,ST0001,201418,671464
3,A01,ST0002,201418,134467
4,A01,ST0001,201413,470040


In [2]:
selloutData.shape

(265, 4)

In [3]:
sortKey = ["REGIONID", "PRODUCTGROUP", "YEARWEEK"]
sortedData = selloutData.sort_values(sortKey, ignore_index=True)
sortedData

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME
0,A01,ST0001,201401,513598
1,A01,ST0001,201402,438251
2,A01,ST0001,201403,420290
3,A01,ST0001,201404,458431
4,A01,ST0001,201405,482381
...,...,...,...,...
260,A01,ST0002,201622,214929
261,A01,ST0002,201623,161983
262,A01,ST0002,201624,171592
263,A01,ST0002,201625,157283


In [4]:
sortedData["MA"] = sortedData.VOLUME.rolling(window=5, center=True, min_periods=1).mean()

In [5]:
sortedData

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME,MA
0,A01,ST0001,201401,513598,457379.666667
1,A01,ST0001,201402,438251,457642.500000
2,A01,ST0001,201403,420290,462590.200000
3,A01,ST0001,201404,458431,473953.000000
4,A01,ST0001,201405,482381,510381.200000
...,...,...,...,...,...
260,A01,ST0002,201622,214929,183080.200000
261,A01,ST0002,201623,161983,182354.200000
262,A01,ST0002,201624,171592,180455.200000
263,A01,ST0002,201625,157283,171836.750000


In [6]:
sortedData.to_csv('./MA_result.csv', index=False)

### 쉬는 시간 동안 위 코드 문제점 찾기!

In [7]:
# ST0001 상품의 마지막 연주차가, ST0002의 처음 연주차랑 행이 이어지다보니 서로 다른 상품의 판매량이 이동평균으로 계산된다.
# 이동평균 구하기 전에 ST0001, ST0002 데이터를 분리해야한다.

### 1. 그룹별로 분리하고
### 2. 이동평균 구하는 함수를 구현

In [1]:
# 1. 데이터를 불러온 후 정렬한다.
import pandas as pd
url = '../exampleCode/dataset/kopo_product_volume.csv'
selloutData = pd.read_csv(url)

sortKey = ["REGIONID", "PRODUCTGROUP", "YEARWEEK"]
sortedData = selloutData.sort_values(sortKey, ignore_index=True)

In [1]:
# 2. 그룹데이터를 만들어 그룹별로 분리시키는 함수를 만든다.
groupKey = ['REGIONID','PRODUCTGROUP']
groupData = sortedData.groupby(groupKey)

In [2]:
len(groupData)

2

In [3]:
groupData.groups

{('A01',
  'ST0001'): Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
             ...
             123, 124, 125, 126, 127, 128, 129, 130, 131, 132],
            dtype='int64', length=133),
 ('A01',
  'ST0002'): Int64Index([133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
             ...
             255, 256, 257, 258, 259, 260, 261, 262, 263, 264],
            dtype='int64', length=132)}

In [4]:
list(groupData.groups)

[('A01', 'ST0001'), ('A01', 'ST0002')]

In [5]:
eachGroup = groupData.get_group(('A01', 'ST0001'))    # groupData.get_group(list(groupData.groups)[0]) 와 같다. # Key를 통해 Value에 접근
eachGroup

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME
0,A01,ST0001,201401,513598
1,A01,ST0001,201402,438251
2,A01,ST0001,201403,420290
3,A01,ST0001,201404,458431
4,A01,ST0001,201405,482381
...,...,...,...,...
128,A01,ST0001,201623,768124
129,A01,ST0001,201624,772706
130,A01,ST0001,201625,786218
131,A01,ST0001,201626,871247


In [6]:
groupData.get_group(('A01', 'ST0002'))    # groupData.get_group(list(groupData.groups)[1]) 과 같다.

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME
133,A01,ST0002,201401,155729
134,A01,ST0002,201402,123875
135,A01,ST0002,201403,114744
136,A01,ST0002,201404,137162
137,A01,ST0002,201405,180516
...,...,...,...,...
260,A01,ST0002,201622,214929
261,A01,ST0002,201623,161983
262,A01,ST0002,201624,171592
263,A01,ST0002,201625,157283


In [7]:
# inplace를 했을 때 warning이 발생. 실제 에러는 아니고 경고이므로 보기 싫으면 이렇게 넣어주면 된다.
import warnings

warnings.filterwarnings(action="ignore")

In [8]:
eachGroup["MA"] = eachGroup.VOLUME.rolling(window=5, center=True, min_periods=1).mean()
eachGroup

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME,MA
0,A01,ST0001,201401,513598,457379.666667
1,A01,ST0001,201402,438251,457642.500000
2,A01,ST0001,201403,420290,462590.200000
3,A01,ST0001,201404,458431,473953.000000
4,A01,ST0001,201405,482381,510381.200000
...,...,...,...,...,...
128,A01,ST0001,201623,768124,773383.400000
129,A01,ST0001,201624,772706,803234.600000
130,A01,ST0001,201625,786218,788871.200000
131,A01,ST0001,201626,871247,794058.000000


In [9]:
# groupData = sortedData.groupby(groupKey)

def groupma(eachGroup):
    # 디버깅 코드
#     eachGroup = groupData.get_group(list(groupData.groups)[0])
    eachGroup["MA"] = eachGroup.VOLUME.rolling(window=5, center=True, min_periods=1).mean()
    
    return eachGroup

In [10]:
# 3. 2에서 구한 함수를 적용해서 이동평균을 구한다.
groupResult = sortedData.groupby(groupKey).apply(groupma)
groupResult

,REGIONID,PRODUCTGROUP,YEARWEEK,VOLUME,MA
0,A01,ST0001,201401,513598,457379.666667
1,A01,ST0001,201402,438251,457642.500000
2,A01,ST0001,201403,420290,462590.200000
3,A01,ST0001,201404,458431,473953.000000
4,A01,ST0001,201405,482381,510381.200000
...,...,...,...,...,...
260,A01,ST0002,201622,214929,183080.200000
261,A01,ST0002,201623,161983,182354.200000
262,A01,ST0002,201624,171592,180455.200000
263,A01,ST0002,201625,157283,171836.750000


In [14]:
groupResult.to_csv('./Test.csv', index=False)